INSTALAÇÃO DE DEPENDÊNCIAS E BIBLIOTECAS QUE SERÃO NECESSÁRIAS

In [ ]:
!pip install pydantic
!pip install pydantic[email]
!pip install fastapi

IMPORTAÇÕES DAS BIBLIOTECAS QUE SERÃO UTILIZADAS NO CÓDIGO

In [20]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

CRIAÇÃO DE UM OBJETO FASTAPI

In [21]:
app = FastAPI()

CRIAÇÃO DA CLASSE USUÁRIO COM VALIDAÇÕES E SERIALIZAÇÃO PERSONALIZADAS PARA OS DADOS

In [ ]:
class User(BaseModel):
    # Isso proíbe que atributos extras sejam adicionados ao modelo
    model_config = {
        "extra": "forbid",
    }

    __users__ = []
    
    # O uso do 'str' é para garantir que o campo seja uma string, 
    # além disso ele oferce um exemplo de um valor válido para essa variável
    name: str = Field(examples=["Example"])
    
    # O uso do 'EmailStr' é para garantir que o campo seja um email
    # O 'Field' é utilizado para adicionar metadados e resrições ao campo, 
    # servindo como uma indicação do formato esperado para esse campo
    email: EmailStr = Field(
        # Ele oferce um exemplo de um valor válido para essa variável
        examples=["user@arjancodes.com"],
        # Aqui é feita uma descrição do que é esperado para esse campo
        description="The email address of the user",
        #O parâmetro Frozen é utilizado para garantir que o objeto não seja alterado após a sua criação
        frozen=True,
    )

    # Criação de uma lista de amigos, com um limite de 500 amigos 
    friends: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of friends"
    )

    # Criação de uma lista de usuários bloqueados, com um limite de 500 usuários
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of blocked users"
    )

    # Criação de um campo de data de registro, com um valor padrão de agora
    # O 'Optional' é utilizado para indicar que o campo é opcional
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description="Signup timestamp", kw_only=True
    )

    # Criação de um campo de identificação, com um valor padrão de um UUID4
    # Esse valor será utilizado para identificar o usuário de forma única
    id: UUID4 = Field(
        default_factory=uuid4, description="Unique identifier", kw_only=True
    )
    
    # A função será responsável por serializar o campo 'id' para o formato 'json'
    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID4) -> str:
        return str(id)

DEFINIÇÃO DE UMA API REST

In [23]:
# Endpoint responsável por retornar a lista de usuários
@app.get("/users", response_model=list[User])
async def get_users() -> list[User]:
    return list(User.__users__)

# Endpoint responsável por criar um novo usuário
@app.post("/users", response_model=User)
async def create_user(user: User):
    User.__users__.append(user)
    return user

# Endpoint responsável por retornar um usuário específico, por meio do seu id
@app.get("/users/{user_id}", response_model=User)
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})


CRIAÇÃO DA FUNÇÃO MAIN (PRINCIPAL)

In [24]:
def main() -> None:
    # Criação de um cliente de teste para testar a aplicação, sem precisar rodar o servidor
    with TestClient(app) as client:
        # Teste de criação de 5 usuários
        for i in range(5):
            response = client.post(
                "/users",
                json={"name": f"User {i}", "email": f"example{i}@arjancodes.com"},
            )

            # Verificação de que o código de resposta é 200
            assert response.status_code == 200
            
            # Verificação de que o nome do usuário é "User {i}"
            assert response.json()["name"] == f"User {i}", (
                "The name of the user should be User {i}"
            )
            
            # Verificação de que o usuário tem um id
            assert response.json()["id"], "The user should have an id"

            # Conversão do json para um objeto do tipo User e validação do modelo
            user = User.model_validate(response.json())

            # Verificação de que o id do usuário é o mesmo do json
            assert str(user.id) == response.json()["id"], "The id should be the same"

            # Verificação de que o timestamp de registro foi definido
            assert user.signup_ts, "The signup timestamp should be set"

            # Verificação de que a lista de amigos está vazia
            assert user.friends == [], "The friends list should be empty"

            # Verificação de que a lista de bloqueados está vazia
            assert user.blocked == [], "The blocked list should be empty"

        # Teste de obtenção da lista de usuários criados anteriormente
        response = client.get("/users")

        # Verificação de que o código de resposta é 200
        assert response.status_code == 200, "Response code should be 200"

        # Verificação de que a lista de usuários tem 5 usuários
        assert len(response.json()) == 5, "There should be 5 users"

        # Criação de um novo usuário
        response = client.post(
            "/users", json={"name": "User 5", "email": "example5@arjancodes.com"}
        )

        # Verificação de que o código de resposta é 200
        assert response.status_code == 200
        
        # Verificação de que o nome do usuário é "User 5"
        assert response.json()["name"] == "User 5", (
            "The name of the user should be User 5"
        )
        
        # Verificação de que o usuário tem um id
        assert response.json()["id"], "The user should have an id"
        
        # Conversão do json para um objeto do tipo User e validação do modelo
        user = User.model_validate(response.json())

        # Verificação de que o id do usuário é o mesmo do json
        assert str(user.id) == response.json()["id"], "The id should be the same"
        # Verificação de que o timestamp de registro foi definido
        assert user.signup_ts, "The signup timestamp should be set"
        # Verificação de que a lista de amigos está vazia
        assert user.friends == [], "The friends list should be empty"
        # Verificação de que a lista de bloqueados está vazia
        assert user.blocked == [], "The blocked list should be empty"

        # Teste de obtenção de um usuário específico, que nesse caso é o último usuário criado
        response = client.get(f"/users/{response.json()['id']}")
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "This should be the newly created user"
        )

        # Teste de obtenção de um usuário específico, que nesse caso é um usuário que não existe
        response = client.get(f"/users/{uuid4()}")
        assert response.status_code == 404
        assert response.json()["message"] == "User not found", (
            "We technically should not find this user"
        )

        # Teste de criação de um usuário com um email inválido
        response = client.post("/users", json={"name": "User 6", "email": "wrong"})
        assert response.status_code == 422, "The email address is should be invalid"

EXECUÇÃO DA FUNÇÃO MAIN 

In [25]:
if __name__ == "__main__":
    main()